In [ ]:
import numpy as np
from skimage import io
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import sys
"push test"
'local_main rename test'
# sys.path.append(r'/Users/CHJ/文稿/无人机sar/yamakawa/pi_data/SAR_program')
import SA_func as sa

## 引数定義
ch = 8 # channel
ad_samp_point = 512  # レーダのレンジ方向サンプリング数
az_n = 2000 #アジマス方向のピクセル数
az_dt =0.01
light_speed = sa.light_speed
df = sa.df
dr = sa.dr
ad_samp_point = sa.ad_samp_point
chirp_rate = sa.chirp_rate
d_tau = df / chirp_rate 
dir_name = ""
add_name = ""
filename = "/Users/CHJ/文稿/无人机sar/yamakawa/pi_data/1206_data/azimuth_1/fft_data"

fft_data = sa.read_fft_data(filename)
data = sa.code_V_convert(fft_data)
raw_data = sa.get_raw_data(data)
index = [700, 1200, 0, 50]
az_s_index = index[0]
az_e_index = index[1]
az_len = az_e_index - az_s_index
rg_s_index = index[2]
rg_e_index = index[3]
rg_len = rg_e_index - rg_s_index
plt.figure(figsize = (12,8))

# ampを抽出,amp_dataはdb化データ
amp = raw_data[0] #raw_data (2000,512)
amp_data = 20 * np.log10(np.abs(amp[az_s_index:az_e_index, rg_s_index:rg_e_index]))

# 创建SOM类
class SOM:
    def __init__(self, n_input, n_output):
        self.n_input = n_input
        self.n_output = n_output
        self.weights = np.random.rand(n_output[0], n_output[1], n_input)

    def train(self, input_data, num_epochs):
        for epoch in range(num_epochs):
            for input_vec in input_data:
                # 找到最佳匹配单元
                bmu_pos = self.get_bmu(input_vec)

                # 更新邻域权重
                self.update_weights(input_vec, bmu_pos, epoch, num_epochs)

    def get_bmu(self, input_vec):
        # 找到距离输入向量最近的单元
        bmu_pos = np.array([0, 0])
        min_dist = np.iinfo(np.int).max
        for x in range(self.n_output[0]):
            for y in range(self.n_output[1]):
                weight_vec = self.weights[x, y, :]
                dist = np.linalg.norm(input_vec - weight_vec)
                if dist < min_dist:
                    min_dist = dist
                    bmu_pos = np.array([x, y])
        return bmu_pos

    def update_weights(self, input_vec, bmu_pos, epoch, num_epochs):
        # 计算邻域函数并更新权重
        eta = self.get_learning_rate(epoch, num_epochs)
        radius = self.get_neighborhood_radius(epoch, num_epochs)
        for x in range(self.n_output[0]):
            for y in range(self.n_output[1]):
                dist = np.linalg.norm(bmu_pos - np.array([x, y]))
                if dist < radius:
                    influence = self.get_influence(dist, radius)
                    weight_diff = eta * influence * (input_vec - self.weights[x, y, :])
                    self.weights[x, y, :] += weight_diff

    def get_learning_rate(self, epoch, num_epochs):
        # 返回时间常数下降的学习率
        return (num_epochs - epoch) / float(num_epochs)

    def get_neighborhood_radius(self, epoch, num_epochs):
        # 返回随时间常数下降的邻域半径
        init_radius = max(self.n_output[0], self.n_output[1]) / 2
        time_constant = num_epochs / np.log(init_radius)
        radius = init_radius * np.exp(-epoch / time_constant)
        return radius

    def get_influence(self, dist, radius):
        # 返回高斯邻域函数
        return np.exp(-dist**2 / (2*radius**2))

# 创建一个二维数据集
data = np.random.rand(100, 2)

# 创建一个SOM对象，设置输入向量大小为2，输出网格大小为10x10
som = SOM(2, (10, 10))

# 训练SOM
som.train(data, 100)

# 可视化训练后的权重向量
plt.imshow(som.weights, interpolation='nearest')
plt.show()

# heatmapを描く
plt.figure(figsize=(12, 8))
sns.heatmap(clustered_image, cmap="jet", vmin=-30, vmax=30) 
name = ["", "range [m]", "azimuth [s]"]
dx = d_tau * light_speed / 2 / 2
dy = 0.01
x_step = int(rg_len / 25) + 1
y_step = int(az_len / 20)
all_font = 20
plt.xticks(np.arange(0, rg_e_index - rg_s_index, step = x_step), np.round(np.arange(rg_s_index * dx, rg_e_index * dx, step = dx * x_step), 2), fontsize = all_font, rotation = 90)
plt.yticks(np.arange(0, az_e_index - az_s_index, step = y_step), np.round(np.arange(az_s_index * dy, az_e_index * dy, step = dy * y_step), 2), fontsize = all_font)
plt.title(name[0])
plt.xlabel(name[1], fontsize = all_font)
plt.ylabel(name[2], fontsize = all_font)
plt.tight_layout()
plt.xlabel("Range")
plt.ylabel("Azimuth")
plt.title("Clustering Result")